In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm
import torch.optim as optim
import numpy as np
import pandas as pd
from tqdm import tqdm
torch.manual_seed(1)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cpu = torch.device('cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [2]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

In [3]:
import torch.nn as nn

class DFDCNet(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers, drop_prob=0.5):
        super(DFDCNet, self).__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        self.mixnet = timm.create_model("mixnet_s", pretrained=True)
        self.mixnet.classifier = Identity()
#         l = 0
#         for param in self.mixnet.parameters():
#             l = l + 1
#             if l > 200:
#                 param.requires_grad = True
                
        self.lstm = nn.LSTM(1536, hidden_dim, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(0.5)
        self.batchnorm = nn.BatchNorm1d(hidden_dim)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_dim, 32)
#         self.fc2 = nn.Linear(64, 32)
#         self.fc3 = nn.Linear(32, 16)
        self.fc4 = nn.Linear(32, output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, hidden):
        batch_size, seqlen, c, h, w = x.size()
        x = x.reshape(batch_size*seqlen, c, h, w).float()
        x = self.mixnet(x)
        x = x.reshape(batch_size, seqlen, x.shape[1])
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        out = self.dropout(lstm_out)
#         out = self.batchnorm(out)
        out = self.fc1(out)
        out = self.elu(out)
#         out = self.fc2(out)
#         out = self.elu(out)
#         out = self.fc3(out)
#         out = self.elu(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        
        out = out.view(batch_size, -1)
        out = out[:,-1]
        return out, hidden
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

In [4]:
#  !ls -ln data_images | head -n 50

In [5]:
for p in np.arange(0, 50):
    if p == 0:
        metadata = torch.load('data_images/000metadata_part_' + str(p) + '.pt', map_location = cpu)
    else:
        metadata_p = torch.load('data_images/000metadata_part_' + str(p) + '.pt', map_location = cpu)
        metadata = pd.concat([metadata, metadata_p])

In [6]:
metadata.n_face.value_counts()

1    104891
2      8879
0      5076
3       308
Name: n_face, dtype: int64

In [7]:
metadata.head()

,label,original,split,n_face
owxbbpjpch.mp4,FAKE,wynotylpnm.mp4,train,1
vpmyeepbep.mp4,REAL,NaN,train,1
fzvpbrzssi.mp4,REAL,NaN,train,1
htorvhbcae.mp4,FAKE,wclvkepakb.mp4,train,1
fckxaqjbxk.mp4,FAKE,vpmyeepbep.mp4,train,1


In [8]:
X = np.array(metadata.index[metadata.n_face == 1])
Y = np.array(1 * (metadata.label[metadata.n_face == 1] == 'REAL'))

In [9]:
X[0:5]

array(['owxbbpjpch.mp4', 'vpmyeepbep.mp4', 'fzvpbrzssi.mp4',
       'htorvhbcae.mp4', 'fckxaqjbxk.mp4'], dtype=object)

In [10]:
Y[0:5]

array([0, 1, 1, 0, 0])

In [11]:
len(X)

104891

In [12]:
len(Y)

104891

In [13]:
n_videos = len(X)
n_videos_train = int(1000 * 80)
n_videos_val = int(100 * (np.floor((n_videos - n_videos_train)/100)))
print(str(n_videos_train) + ' for training')
print(str(n_videos_val) + ' for validation')

80000 for training
24800 for validation


In [14]:
from torch.utils import data
class Dataset(data.Dataset):
    'Characterizes a dataset for PyTorch'
    def __init__(self, list_IDs, labels):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]
        # Load data and get label
        X = torch.load('data_images/1face_X_'+ ID + '.pt', map_location = cpu)
        y = self.labels[index]

        return X[[0,2,4,6,8]], y

In [15]:
from torch.utils.data import TensorDataset, DataLoader
dataset = Dataset(X[0:n_videos_train + n_videos_val], Y[0:n_videos_train + n_videos_val])
train_data, val_data = torch.utils.data.random_split(dataset, [n_videos_train, n_videos_val])
train_batch_size = 40
val_batch_size = 25
train_loader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size, num_workers = 8)
val_loader = DataLoader(val_data, shuffle=True, batch_size=val_batch_size, num_workers = 8)

In [16]:
input_size = 512
output_size = 1
hidden_dim = 512
n_layers = 2

model = DFDCNet(input_size, output_size, hidden_dim, n_layers)
model.to(device)

DFDCNet(
  (mixnet): EfficientNet(
    (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16, bias=False)
          (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU(inplace=True)
          (conv_pw): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (1): Sequential(
        (0): InvertedResidual(
          (conv_pw): MixedConv2d(
            (0): Conv2d(8, 48, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): Conv2d(8,

In [17]:
param_optimizer = list(model.named_parameters())

train_criterion = nn.BCELoss()
val_criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [18]:
# scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=500, verbose=True)

In [19]:
torch.cuda.empty_cache() 

In [20]:
! rm log_training.log

In [21]:
import logging
logging.basicConfig(filename = 'log_training.log',level = logging.INFO)


In [22]:

from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 23:42:18


In [23]:
epochs = 5
counter = 0
print_every = 1000
clip = .5
valid_loss_min = .15
val_loss = torch.tensor(np.Inf)
model.train()
for i in range(epochs):
    h = model.init_hidden(train_batch_size)
    for inputs, labels in tqdm(train_loader):

        counter += 1
        h = tuple([e.data for e in h])
        inputs, labels = inputs.to(device), labels.to(device)
        model.zero_grad()
        output, h = model(inputs, h)
        loss = train_criterion(output.squeeze(), labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        torch.cuda.empty_cache() 
#         print(counter)
        if counter % 100 == 0:
            logging.info("Time: {}...".format(datetime.now().strftime("%H:%M:%S")) + 
                         "Epoch: {}/{}...".format(i+1, epochs) +  
                         "Step: {}...".format(counter) +
                         "Loss: {:.6f}...".format(loss.item())) 
        
        if counter%print_every == 0:
            logging.info(str(counter))
            val_h = model.init_hidden(val_batch_size)
            val_losses = []
            model.eval()
            for inp, lab in tqdm(val_loader):
                
                val_h = tuple([each.data for each in val_h])
                inp, lab = inp.to(device), lab.to(device)
                out, val_h = model(inp, val_h)
                val_loss = val_criterion(out.squeeze(), lab.float())
                val_losses.append(val_loss.item())
            model.train()
            print("Time: {}...".format(datetime.now().strftime("%H:%M:%S")) + 
                  "Epoch: {}/{}...".format(i+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.6f}...".format(loss.item()),
                  "Val Loss: {:.6f}".format(np.mean(val_losses)))
            logging.info("Time: {}...".format(datetime.now().strftime("%H:%M:%S")) + 
                         "Epoch: {}/{}...".format(i+1, epochs) +  
                         "Step: {}...".format(counter) +
                         "Loss: {:.6f}...".format(loss.item()) +
                         "Val Loss: {:.6f}".format(np.mean(val_losses)))
            if np.mean(val_losses) <= valid_loss_min:
                torch.save(model.state_dict(), './model_1face_unfroze.pt')
                print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                logging.info('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,np.mean(val_losses)))
                valid_loss_min = np.mean(val_losses)
                torch.cuda.empty_cache() 
    scheduler.step(val_loss.item())


 24%|██▍       | 243/992 [04:19<16:23,  1.31s/it]


 54%|█████▍    | 537/992 [10:00<15:10,  2.00s/it]


 84%|████████▍ | 837/992 [17:39<03:57,  1.53s/it]


 50%|█████     | 1000/2000 [45:16<114:14:26, 411.27s/it]

Time: 00:27:34...Epoch: 1/5... Step: 1000... Loss: 0.075605... Val Loss: 0.133982
Validation loss decreased (0.150000 --> 0.133982).  Saving model ...


...

 45%|████▍     | 443/992 [25:31<11:47,  1.29s/it]


 67%|██████▋   | 664/992 [28:46<05:27,  1.00it/s]


 90%|█████████ | 896/992 [32:14<02:00,  1.26s/it]


100%|██████████| 2000/2000 [1:45:44<00:00, 607.43s/it]

Time: 07:28:26...Epoch: 5/5... Step: 10000... Loss: 0.083125... Val Loss: 0.062692
Validation loss decreased (0.063347 --> 0.062692).  Saving model ...


# Reference
* https://github.com/ronghanghu/pytorch-gve-lrcn/blob/master/models/pretrained_models.py